> 일단 요런걸 찾긴했지만..
>> 개인적으로는 코드 분석하면서 씹고 뜯고 맛보고 즐기고 싶은 취향(?)이 있다..

[참고링크](https://velog.io/@hwanython/%EB%84%A4%EC%9D%B4%EB%B2%84-%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0-%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A5%BC-%EA%B0%80%EC%A7%80%EA%B3%A0-%EA%B0%90%EC%84%B1%EC%9D%84-%EB%B6%84%EC%84%9D)</br>

~근데 진심 7/13 개인 저녁일정은 에바지 ㅠ~

# 8-1. 프로젝트 : 네이버 영화리뷰 감성분석 도전하기
네이버 영화의 댓글을 모아 구성된 `Naver sentiment movie corpus`</br>

```shell
# 노드 기준 데이터 다운로드 없이 Cloud shell에서 파일 링킹
$ ln -s ~/data/*.txt ~/aiffel/sentiment_classification/data
```

[Github(Naver sentiment movie corpus)](https://github.com/e9t/nsmc)</br>

**평가문항**</br>
    **상세기준**</br>

---

1. 다양한 방법으로 Text Classification 태스크를 성공적으로 구현하였다.
    > 3가지 이상의 모델이 성공적으로 시도됨
2. gensim을 활용하여 자체학습된 혹은 사전학습된 임베딩 레이어를 분석하였다.
    > gensim의 유사단어 찾기를 활용하여 자체학습한 임베딩과 사전학습 임베딩을 비교 분석함
3. 한국어 Word2Vec을 활용하여 가시적인 성능향상을 달성했다.
    > 네이버 영화리뷰 데이터 감성분석 정확도를 85% 이상 달성함

## 라이브러리 버전 확인

In [ ]:
import pandas
import konlpy
import gensim

print(pandas.__version__)
print(konlpy.__version__)
print(gensim.__version__)

## 1) 데이터 준비와 확인

In [ ]:
import pandas as pd

# 데이터를 읽어봅시다.; 노드 기준 예시
train_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_train.txt')
test_data = pd.read_table('~/aiffel_sentiment_classification/data/rating_test.txt')

train_data.head()

## 2) 데이터로더 구성

실습 때 다뤘던 IMDB 데이터셋은 텍스트를 가공해 `imdb.data_loader()` 메서드를 통해 숫자 인덱스로 변환된 텍스트와 `word_to_index` 딕셔너리까지 친절하게 제공하였음</br>
그러나 이번 nsmc 데이터셋은 전혀 가공되지 않은 텍스트 파일로 구성되어 있다.</br>

이것을 읽어서 `imdb.data_loader()`와 동일하게 동작하는 자신만의 `data_loader`를 만들어보는 것으로 시작한다.</br>
`data_loader` 내부에서는 다음을 수행해야 한다.</br>

- 데이터의 중복제거
- NaN 결측치 제거
- 한국어 토크나이저로 토큰화
- 불용어(Stopwords) 제거
- 사전 `word_to_index` 구성
- 텍스트 스트링을 사전 인덱스 스트링으로 변환
- `X_train`, `y_train`, `X_test`, `y_test`, `word_to_index` 리턴

In [ ]:
from konlpy.tag import Mecab
import numpy as np
from collections import Counter

tokenizer = Mecab()
stopwords = ['의', '가', '이', '은', '들', 
             '는', '좀', '잘', '걍', '과', 
             '도', '를', '으로', '자', '에', 
             '와', '한', '한다']

def load_data(train_data, test_data, num_words=num_words):
    # [[YOUR CODE]]
    pass

X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)

<details>
<summary>예시 답안</summary>

<div markdown="1">
    ```python

        from konlpy.tag import Mecab
        import numpy as np
        from collections import Counter

        tokenizer = Mecab()
        stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

        def load_data(train_data, test_data, num_words=10000):
            train_data.drop_duplicates(subset=['document'], inplace=True)
            train_data = train_data.dropna(how = 'any') 
            test_data.drop_duplicates(subset=['document'], inplace=True)
            test_data = test_data.dropna(how = 'any') 
            
            X_train = []
            for sentence in train_data['document']:
                temp_X = tokenizer.morphs(sentence) # 토큰화
                temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
                X_train.append(temp_X)

            X_test = []
            for sentence in test_data['document']:
                temp_X = tokenizer.morphs(sentence) # 토큰화
                temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
                X_test.append(temp_X)
            
            words = np.concatenate(X_train).tolist()
            counter = Counter(words)
            counter = counter.most_common(10000-4)
            vocab = ['', '', '', ''] + [key for key, _ in counter]
            word_to_index = {word:index for index, word in enumerate(vocab)}
                
            def wordlist_to_indexlist(wordlist):
                return [word_to_index[word] if word in word_to_index else word_to_index[''] for word in wordlist]
                
            X_train = list(map(wordlist_to_indexlist, X_train))
            X_test = list(map(wordlist_to_indexlist, X_test))
                
            return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index
            
        X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)
    ```
</div>
</details>

In [ ]:
index_to_word = {index:word for word, index in word_to_index.items()}

In [ ]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다.
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다.
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK'] for word in sentence.split()]

# 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode 해주는 함수입니다.
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다.
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  # [1:]를 통해 <BOS>를 제외

# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다.
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

## 3) 모델 구성을 위한 데이터 분석 및 가공

- 데이터셋 내 문장 길이 분포
- 적절한 최대 문장 길이 지정
- keras.preprocessing.sequence.pad_sequences을 활용한 패딩 추가

## 4) 모델 구성 및 validation set 구성

모델은 3가지 이상 다양하게 구성하여 실험해 보세요

## 5) 모델 훈련 개시

## 6) Loss, Accuracy 그래프 시각화

## 7) 학습된 Embedding 레이어 분석

## 8) 한국어 Word2Vec 임베딩 활용하여 성능 개선

- 한국어 Word2Vec은 `/data` 폴더 안에 있는 `word2vec_ko.model`을 활용하세요.
- 한국어 Word2Vec을 활용할 때는 `load_word2vec_format()` 형태가 아닌 `load()` 형태로 모델을 불러와주세요.
- 또한 모델을 활용할 때는 아래 예시와 같이 `.wv`를 붙여서 활용합니다.

<details>
<summary>예시 코드</summary>
<div markdown="1">
```python
    # 예시 코드
    from gensim.models.keyedvectors import Word2VecKeyedVectors
    word_vectors = Word2VecKeyedVectors.load(word2vec_file_path)
    vector = word_vectors.wv['끝']
```
</div>
</details>

- 좀 더 자세한 활용법에 대해선 다음 링크들을 참조해주세요.
    - [참고 링크1](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#storing-and-loading-models)
    - [참고 링크2](https://radimrehurek.com/gensim/models/keyedvectors.html)